# Taufiq July Kurniawan_Posttest 8
# 2009106138

Import Library yang di butuhkan

In [173]:
from google.colab import drive
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

##### Read Dataset

In [174]:
drive.mount('/content/drive/')
# load dataset
df = pd.read_csv('netflix_titles.csv')
df

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


##### Membuat Fungsi Callback

In [175]:
 
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.95):
      print("\n Accuracy Telah melebihi > 95%")
      self.model.stop_training = True
 
callbacks = myCallback()

In [176]:
# encoding
enc = pd.get_dummies(df.show_id)
df = pd.concat([df, enc], axis = 1)
df = df.drop(columns = 'show_id')
 
df

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,...,s990,s991,s992,s993,s994,s995,s996,s997,s998,s999
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,...,0,0,0,0,0,0,0,0,0,0
1,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries",...,0,0,0,0,0,0,0,0,0,0
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",...,0,0,0,0,0,0,0,0,0,0
3,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV",...,0,0,0,0,0,0,0,0,0,0
4,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers",...,0,0,0,0,0,0,0,0,0,0
8803,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies",...,0,0,0,0,0,0,0,0,0,0
8804,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",...,0,0,0,0,0,0,0,0,0,0
8805,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies",...,0,0,0,0,0,0,0,0,0,0


In [177]:
# split dataframe
review = df['title'].values
label = df[['s1', 's10']].values

##### Pembagian data

In [178]:
text = df['title'].values
y = df[['s1', 's10']].values
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size = 0.2)

##### Menggunakan fungsi tokenizer

In [179]:
# tokenizer
tokenizer = Tokenizer(num_words = 350617, oov_token = '-')
tokenizer.fit_on_texts(text_train)
tokenizer.fit_on_texts(text_test)
 
# sequences
seq_train = tokenizer.texts_to_sequences(text_train)
seq_test = tokenizer.texts_to_sequences(text_test)
 
# padding
padding_train = pad_sequences(seq_train, maxlen = 300, padding = 'post', truncating = 'post')
 
padding_test = pad_sequences(seq_test, maxlen = 300, padding = 'post', truncating = 'post')

##### Membuat Model Sequensial

In [180]:
# embedding
model = Sequential([layers.Embedding(35017, 64, input_length = 300),
                    layers.LSTM(64, dropout = 0.1),
                    layers.Dense(128, activation = 'relu'),
                    layers.Dense(64, activation = 'relu'),
                    layers.Dense(2, activation = 'sigmoid')
                    ])
 
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 300, 64)           2241088   
                                                                 
 lstm_9 (LSTM)               (None, 64)                33024     
                                                                 
 dense_41 (Dense)            (None, 128)               8320      
                                                                 
 dense_42 (Dense)            (None, 64)                8256      
                                                                 
 dense_43 (Dense)            (None, 2)                 130       
                                                                 
Total params: 2,290,818
Trainable params: 2,290,818
Non-trainable params: 0
_________________________________________________________________


##### Fungsi Accuracy

In [181]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
 
history = model.fit(padding_train, 
                    y_train, 
                    batch_size = 64, 
                    epochs = 35,
                    validation_data = (padding_test, y_test),
                    verbose = 2,
                    callbacks = [callbacks]
                    )

Epoch 1/35
111/111 - 37s - loss: 0.0741 - accuracy: 0.5029 - val_loss: 1.1886e-05 - val_accuracy: 1.0000 - 37s/epoch - 329ms/step
Epoch 2/35
111/111 - 35s - loss: 0.0016 - accuracy: 0.2559 - val_loss: 9.5147e-05 - val_accuracy: 1.0000 - 35s/epoch - 312ms/step
Epoch 3/35

 Accuracy Telah melebihi > 95%
111/111 - 34s - loss: 0.0015 - accuracy: 0.9999 - val_loss: 1.4091e-04 - val_accuracy: 1.0000 - 34s/epoch - 305ms/step
